# Projet maison n° 3

In [529]:
# imports
import pandas as pd
import sys
from os import listdir
import re
import numpy as np

In [530]:
colnames=['name', 'gender', 'births', 'year'] 
df1 = pd.read_csv('names/yob1880.txt', names=colnames, header=None)
df2 = pd.read_csv('names/yob1881.txt', names=colnames, header=None)
dir_path = '/Users/scotto/Documents/TélécomParis/KitBigData/Kit-Big-Data/session3/names'

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [574]:
# names us
def df_names_us():
    df = pd.DataFrame()
    
    for file in listdir(dir_path):
        date = re.sub(r'([a-z]+)([0-9]+)(.[a-z]+)', r'\2', file)
        df_new = pd.read_csv(dir_path+ ('/') +file, names=colnames, header=None, encoding = 'unicode_escape')
        df_new = df_new.assign(year = date)
        #print(df_new)
        df = pd.concat([df,df_new])

    colnames2=['year', 'name', 'gender', 'births'] 
    df = df[colnames2]
    df = df.sort_values('year')
    df = df.dropna()
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    return df

In [575]:
df_names_us()

,year,name,gender,births
0,1880,Johnie,M,18.0
1,1880,Dee,M,20.0
2,1880,Dallas,M,20.0
3,1880,Alphonse,M,20.0
4,1880,Allan,M,20.0
...,...,...,...,...
2052776,2021,Dansby,M,32.0
2052777,2021,Darvin,M,32.0
2052778,2021,Dayson,M,32.0
2052779,2021,Dio,M,32.0


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [576]:
# names fr
def df_names_fr():

    path = '/Users/scotto/Documents/TélécomParis/KitBigData/Kit-Big-Data/session3/nat2021.csv'
    df = pd.DataFrame()
    
    # columns modification
    colnames=['gender', 'name', 'year', 'births'] 
    df = pd.read_csv(path , sep = ';', header=None, encoding = 'unicode_escape', names = colnames, keep_default_na = False)
    colnames2 = ['year', 'name', 'gender', 'births'] 
    df = df[colnames2]
    
    # gender mapping
    mapping = { 2: 'F', 1 : 'M', '1': 'M'}
    df['gender'] = df['gender'].map(mapping)
    
    
    # sorting
    df = df.sort_values(by=['year', 'gender', 'births','name'], ascending = [True, True, False, True] )

    # drop unusable data
    df= df.dropna()
    df.drop(df.loc[df['name'] == '_PRENOMS_RARES'].index, inplace=True)
    df.drop(df.loc[df['name'].str.len() < 2].index, inplace=True)
    df.drop(df.loc[df['year'] == 'XXXX'].index, inplace=True)
    
    
    
    #df = df.drop(df[df['births'].str.contains('nom', na=False)].index)
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    
    #maj problem
    df['name'] = df["name"].str.title()

    return df


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [577]:
def df_taux_change(devises):
    
    # load data
    path = '/Users/scotto/Documents/TélécomParis/KitBigData/Kit-Big-Data/session3/Webstat_Export.csv'
    df = pd.read_csv(path, sep=';')
    
    # select only ISO column names
    iso = [re.search(r'\([A-Z]{3}\)', word) for word in df.columns]
    for i in range(len(iso)):
        if iso[i]!= None:
            iso[i] = iso[i].group(0).replace('(','').replace(')','')
    iso[0] = 'date'            
    df.columns=iso

    # set date column as index
    
    df.drop(index=[0,1,2,3,4], axis=0, inplace=True) #supprime les lignes sans dates
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # return only required columns
    df = df[devises]
    
    # format df to contain float
    df = df.apply(lambda x: x.str.replace(',','.'))
    df = df.apply(lambda x: x.replace('-',np.NaN) )
    df=df.astype('float')
    
    # remove null values
    df.dropna(inplace=True)
    
    # sort datetime
    df.sort_index(inplace=True)
    
    return df

### Tests

In [578]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [579]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [581]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... /var/folders/2t/_v7vz8g96_16z98yf83c6xfm0000gn/T/ipykernel_9192/2568837283.py:16: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  df = df_names_fr()
ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 16.303s

OK
